In [8]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [4]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [5]:
df = pd.read_csv("all_data_clean.csv",header=None,
                 names=['timestamp','accel_x','accel_y','accel_z','gyro_x','gyro_y','gyro_z','label'])
df = df.dropna()
df = df.reset_index(drop=True)
df = df[1:]
df.shape

C:\Users\siyua\AppData\Local\Temp\ipykernel_10452\3893055260.py:1: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_data_clean.csv",header=None,


(125699, 8)

In [6]:
features = ['accel_x','accel_y','accel_z','gyro_x','gyro_y','gyro_z']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])
window_size = 200
step_size = 2

X = []
y = []

for start in range(0,len(df) - window_size, step_size):
    end = start + window_size
    window = df.iloc[start:end]
    label = window['label'].mode()[0]
    
    X.append(window[features].values)
    y.append(label)

X = np.array(X)
y = np.array(y)
print("window data shape",X.shape,y.shape)

window data shape (62750, 200, 6) (62750,)


In [9]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

print("Label classes:", encoder.classes_)

Label classes: ['falling' 'sitting' 'standing' 'walking']


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size = 0.2, random_state=42, stratify=y_cat
)

In [14]:
#define 1D CNN Model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,MaxPooling1D,LSTM,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
   Conv1D(64, 3, activation='relu', input_shape=(window_size, 6)),
    BatchNormalization(),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_cat.shape[1], activation='softmax')
])
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer= optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 198, 64)           1216      
                                                                 
 batch_normalization_2 (Batc  (None, 198, 64)          256       
 hNormalization)                                                 
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 97, 128)           24704     
                                                                 
 batch_normalization_3 (Batc  (None, 97, 128)          512       
 hNormalization)                                                 
                                                      

In [12]:
#Train the model
with tf.device('/GPU:0'):
    history = model.fit(X_train,y_train,epochs=100,batch_size=64,
                    validation_data=(X_test,y_test))

Epoch 1/100
785/785 [==============================] - 14s 4ms/step - loss: 1.0961 - accuracy: 0.5155 - val_loss: 0.8184 - val_accuracy: 0.6646
Epoch 2/100
785/785 [==============================] - 3s 4ms/step - loss: 0.5816 - accuracy: 0.7780 - val_loss: 0.7700 - val_accuracy: 0.7089
Epoch 3/100
785/785 [==============================] - 3s 4ms/step - loss: 0.3494 - accuracy: 0.8730 - val_loss: 0.5533 - val_accuracy: 0.7960
Epoch 4/100
785/785 [==============================] - 3s 4ms/step - loss: 0.2366 - accuracy: 0.9135 - val_loss: 0.4007 - val_accuracy: 0.8584
Epoch 5/100
785/785 [==============================] - 3s 4ms/step - loss: 0.1782 - accuracy: 0.9367 - val_loss: 0.4361 - val_accuracy: 0.8583
Epoch 6/100
785/785 [==============================] - 3s 4ms/step - loss: 0.1434 - accuracy: 0.9494 - val_loss: 0.3923 - val_accuracy: 0.8741
Epoch 7/100
785/785 [==============================] - 3s 4ms/step - loss: 0.1241 - accuracy: 0.9557 - val_loss: 0.4718 - val_accuracy: 0.864

In [13]:
#Evaluate the model
loss, acc = model.evaluate(X_test,y_test)
print("Test accuracy",acc)

393/393 [==============================] - 1s 2ms/step - loss: 0.9296 - accuracy: 0.8889
Test accuracy 0.8889243006706238


In [33]:
model2 = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(window_size, 6)),
    MaxPooling1D(2),
    Dropout(0.3),
    LSTM(64 ,return_sequences=False),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dense(y_cat.shape[1], activation='softmax')
])

optimizer = Adam(learning_rate=0.0005)
model2.compile(optimizer= optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 198, 64)           1216      
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 99, 64)           0         
 g1D)                                                            
                                                                 
 dropout_19 (Dropout)        (None, 99, 64)            0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                      

In [34]:
#model2 trianing 
with tf.device('/GPU:0'):
    history = model2.fit(X_train,y_train,epochs=100,batch_size=64,
                    validation_data=(X_test,y_test))

Epoch 1/100
785/785 [==============================] - 8s 8ms/step - loss: 1.2688 - accuracy: 0.3918 - val_loss: 1.3441 - val_accuracy: 0.4098
Epoch 2/100
785/785 [==============================] - 6s 8ms/step - loss: 1.0893 - accuracy: 0.5152 - val_loss: 1.0103 - val_accuracy: 0.5810
Epoch 3/100
785/785 [==============================] - 7s 8ms/step - loss: 0.9794 - accuracy: 0.5854 - val_loss: 0.9492 - val_accuracy: 0.5925
Epoch 4/100
785/785 [==============================] - 6s 8ms/step - loss: 0.9152 - accuracy: 0.6167 - val_loss: 0.8458 - val_accuracy: 0.6415
Epoch 5/100
785/785 [==============================] - 6s 8ms/step - loss: 0.8724 - accuracy: 0.6353 - val_loss: 0.8492 - val_accuracy: 0.6446
Epoch 6/100
785/785 [==============================] - 6s 8ms/step - loss: 0.8475 - accuracy: 0.6475 - val_loss: 0.7723 - val_accuracy: 0.6767
Epoch 7/100
785/785 [==============================] - 6s 8ms/step - loss: 0.8229 - accuracy: 0.6569 - val_loss: 0.7286 - val_accuracy: 0.6972

In [35]:
loss, acc = model2.evaluate(X_test,y_test)
print("Test accuracy",acc)

393/393 [==============================] - 1s 3ms/step - loss: 0.2530 - accuracy: 0.9034
Test accuracy 0.9034262895584106


In [ ]:
#convert to TFLITE
